In [5]:
# 0. ── Imports ───────────────────────────────────────────────────────────────˜
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn.feature_selection import RFECV
from sklearn.metrics import r2_score, mean_squared_error

# 1. ── Load your imputed V‑Dem subset ────────────────────────────────────────
df = pd.read_csv("../temp/df_imputed.csv")
print("Data shape:", df.shape)
print("Any missing values? ", df.isna().sum().sum(), " (should be 0)")

# 2. ── Define target & predictors ──────────────────────────────────────────
target = "v2x_corr"

# Drop non‑predictors: text IDs, year, numeric ID
drop_cols = ["country_name", "country_text_id", "country_id", "year", target]

# Grab all remaining numeric columns as predictors
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
predictors = [c for c in numeric_cols if c not in drop_cols]

print("Using", len(predictors), "numeric predictors.")

# 3. ── (Optional) Log‑transform GDP per capita ──────────────────────────────
# If you want to include a log of GDP per capita instead of its raw value:
if "e_gdppc_imputed_rf" in predictors:
    df["log_gdppc"] = np.log(df["e_gdppc_imputed_rf"] + 1)
    predictors = [p for p in predictors if p != "e_gdppc_imputed_rf"] + ["log_gdppc"]
    print("Replaced e_gdppc_imputed_rf with log_gdppc.")

# 4. ── Standardize predictors ───────────────────────────────────────────────
scaler = StandardScaler()
df[predictors] = scaler.fit_transform(df[predictors])

# 5. ── Train/Test split ────────────────────────────────────────────────────
X = df[predictors]
y = df[target]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)

# 6. ── Baseline OLS (Statsmodels) ──────────────────────────────────────────
X1_train = sm.add_constant(X_train)
ols_baseline = sm.OLS(y_train, X1_train).fit()
print("\n=== Baseline OLS Summary ===")
print(ols_baseline.summary())


# 7. ── Backward Elimination by p‐value (fixed) ──────────────────────────────
def backward_elimination(X, y, sl=0.05):
    X_be = sm.add_constant(X.copy())
    while True:
        model = sm.OLS(y, X_be).fit()
        pvals = model.pvalues.drop(labels="const", errors="ignore")
        max_p = pvals.max()
        if max_p > sl:
            drop_var = pvals.idxmax()
            X_be = X_be.drop(columns=drop_var)
        else:
            break
    return model


ols_be = backward_elimination(X_train, y_train)
print("\n=== Backward Elim OLS Summary ===")
print(ols_be.summary())

# 8. ── Recursive Feature Elimination with CV ────────────────────────────────
lr = LinearRegression()
rfecv = RFECV(estimator=lr, cv=5, scoring="r2")
rfecv.fit(X_train, y_train)

selected_rfecv = X_train.columns[rfecv.support_].tolist()
print(f"\nRFECV selected {len(selected_rfecv)} features:")
print(selected_rfecv)

# 9. ── LassoCV (L1 regularization) ─────────────────────────────────────────
lasso_cv = LassoCV(cv=5, random_state=0).fit(X_train, y_train)
lasso_coefs = pd.Series(lasso_cv.coef_, index=X_train.columns)
print("\nNon‑zero Lasso coefficients:")
print(lasso_coefs[lasso_coefs != 0])


# 10. ── Model evaluation on test set ───────────────────────────────────────
def evaluate(model, X_t, y_t, sm_model=False):
    # 1) Get predictions
    if sm_model:
        exog_names = list(model.model.exog_names)
        has_int = any(n in exog_names for n in ("const", "Intercept"))
        if has_int:
            dropers = [n for n in ("const", "Intercept") if n in exog_names]
            pred_cols = [n for n in exog_names if n not in dropers]
            X_pred = sm.add_constant(X_t[pred_cols], has_constant="add")
        else:
            pred_cols = exog_names
            X_pred = X_t[pred_cols]
        preds = model.predict(X_pred)
    else:
        preds = model.predict(X_t)

    # 2) Compute metrics
    r2 = r2_score(y_t, preds)
    rmse = mean_squared_error(y_t, preds, squared=False)

    # 3) Print with built-in round()
    print("  R²:   ", round(r2, 3))
    print("  RMSE: ", round(rmse, 3))


print("\n--- Test-set performance ---")
print("Baseline OLS:")
evaluate(ols_baseline, X_test, y_test, sm_model=True)

print("\nBackward‑Elim OLS:")
evaluate(ols_be, X_test, y_test, sm_model=True)

print("\nRFECV LinearRegression:")
model_rf = LinearRegression().fit(X_train[selected_rfecv], y_train)
evaluate(model_rf, X_test[selected_rfecv], y_test)

print("\nLassoCV:")
evaluate(lasso_cv, X_test, y_test)

# 11. ── Final diagnostics for your chosen model (e.g. ols_be) ─────────────
resid = ols_be.resid
fitted = ols_be.fittedvalues

# Residuals vs Fitted
plt.scatter(fitted, resid, alpha=0.5)
plt.axhline(0, linestyle="--", color="k")
plt.xlabel("Fitted values")
plt.ylabel("Residuals")
plt.title("Residuals vs Fitted (Backward‑Elim OLS)")
plt.show()

# QQ‑plot of residuals
sm.qqplot(resid, line="45")
plt.title("QQ‑Plot of Residuals")
plt.show()

# Influence plot
sm.graphics.influence_plot(ols_be, criterion="cooks")
plt.show()

# 12. ── Coefficient table of final model ───────────────────────────────────
coef_table = pd.DataFrame(
    {
        "coef": ols_be.params,
        "std_err": ols_be.bse,
        "p_value": ols_be.pvalues,
        "conf_low": ols_be.conf_int()[0],
        "conf_high": ols_be.conf_int()[1],
    }
)
print("\n=== Final Coefficient Table (Backward‑Elim OLS) ===")
print(coef_table)

Data shape: (9129, 363)
Any missing values?  0  (should be 0)
Using 358 numeric predictors.
Replaced e_gdppc_imputed_rf with log_gdppc.

=== Baseline OLS Summary ===
                            OLS Regression Results                            
Dep. Variable:               v2x_corr   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                 1.289e+04
Date:                Mon, 21 Apr 2025   Prob (F-statistic):               0.00
Time:                        13:51:53   Log-Likelihood:                 21257.
No. Observations:                7303   AIC:                        -4.196e+04
Df Residuals:                    7025   BIC:                        -4.004e+04
Df Model:                         277                                         
Covariance Type:            nonrobust                                         
                                       coef 

TypeError: got an unexpected keyword argument 'squared'